## Задание 1

Представьте, что вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:

В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета.
Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
#импортируем библиотеки

In [2]:
#считываем файлы 
reg = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-streltsova-31/shared/problem1-reg_data.csv', sep =';')
auth = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-streltsova-31/shared/problem1-auth_data.csv', sep =';')

In [3]:
#посмотрим на размер датасетов
reg.shape

(1000000, 2)

In [4]:
auth.shape

(9601013, 2)

In [5]:
#соединяем два датасета с датой регистрации и авторизациями
full = auth.merge(reg, how = 'left', on = 'uid')

In [6]:
full.head()

,auth_ts,uid,reg_ts
0,911382223,1,911382223
1,932683089,2,932683089
2,932921206,2,932683089
3,933393015,2,932683089
4,933875379,2,932683089


In [7]:
full.uid.nunique()

1000000

In [8]:
full.isna().sum()

auth_ts    0
uid        0
reg_ts     0
dtype: int64

In [9]:
full.dtypes

auth_ts    int64
uid        int64
reg_ts     int64
dtype: object

In [10]:
full.loc[full.duplicated()]

,auth_ts,uid,reg_ts


Пропущенных значений и дубликатов нет, данные о 1 млн уникальных пользователей. Данные о регистрации и авторизайии находятся в формате Unix. Переведем их в привычный формат даты

In [11]:
full['reg_ts'] = pd.to_datetime(full['reg_ts'], unit='s').dt.date.astype('datetime64')
full['auth_ts'] = pd.to_datetime(full['auth_ts'], unit='s').dt.date.astype('datetime64')

In [12]:
full.dtypes

auth_ts    datetime64[ns]
uid                 int64
reg_ts     datetime64[ns]
dtype: object

In [13]:
full['reg_ts'].min()

Timestamp('1998-11-18 00:00:00')

In [14]:
full['auth_ts'].max()

Timestamp('2020-09-23 00:00:00')

Датасет очень большой, более 20 лет включено в датасет.
Напишем функцию для расчета retention

In [27]:
def retention_rate_table(reg, auth, start, end, days):
    
    """функция для подсчета retention по дням, 
принимает на вход датасеты с ID и датой регистрации, ID и даты авторизаций, даты начала 
периода и конца периода, а также количество дней для расчета retention"""
    
    #cчитываем данные    
    reg_df = pd.read_csv(reg, sep=';')
    auth_df = pd.read_csv(auth, sep=';')
    
      
    #переводим unix в привычный форматы даты
    reg_df['reg_ts'] = pd.to_datetime(reg_df['reg_ts'], unit='s').dt.date.astype('datetime64')
    auth_df['auth_ts'] = pd.to_datetime(auth_df['auth_ts'], unit='s').dt.date.astype('datetime64')
    
    #оставляем только нужный период
    reg_df = reg_df.query('reg_ts >= @start & reg_ts <= @end')
    auth_df = auth_df.query('auth_ts >= @start')
    
    #соединяем датасеты
    full = auth_df.merge(reg_df, how = 'left', on = 'uid')
       
    #добавляем информацию о периоде  
    
    full['period'] = (full['auth_ts'] - full['reg_ts']).dt.days
    
    #переименовываем столбец с датой регистрации - это когорты
    
    full = full.rename(columns = {'reg_ts': 'cohort_date'})
    
    #отфильтровываем период по нужному нам количеству дней для расчета и делаем сводную
    
    cohort_pivot = full.query('period <= @days').pivot_table(columns = 'period', index = 'cohort_date',
                           values = 'uid', aggfunc = 'nunique')
        
    #делим столбцы на первое значение, чтобы получить процентное соотношение
    cohort_percent = cohort_pivot.divide(cohort_pivot.iloc[:,0], axis=0).mul(100).round(2)
    
    #добавляем кол-во пользователей в когорте
    users = full.groupby('cohort_date').agg({'uid':'nunique'})
    cohort_percent = cohort_percent.merge(users, on='cohort_date', how='left').rename(columns={'uid': 'users'})
    cohort_percent = cohort_percent.reset_index().set_index(['cohort_date', 'users'])
     
    return cohort_percent
    

In [28]:
#тестируем работу функции

reg1= '/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-streltsova-31/shared/problem1-reg_data.csv'
auth1= '/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-streltsova-31/shared/problem1-auth_data.csv'
start = "2020-03-01"
end = "2020-03-15"
days = 15

retention_rate_table (reg1, auth1, start, end, days)

,,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
cohort_date,users,,,,,,,,,,,,,,,,
2020-03-01,1172,100.0,1.96,4.44,4.86,5.20,4.61,6.83,5.20,4.69,4.61,5.20,4.18,5.80,4.18,3.67,5.12
2020-03-02,1173,100.0,1.36,3.67,4.18,4.94,6.73,6.05,6.31,4.86,5.29,5.12,5.71,4.94,5.12,3.67,3.92
2020-03-03,1176,100.0,2.38,2.98,4.85,4.34,5.36,7.48,4.76,4.51,5.70,5.02,4.68,5.27,5.36,4.34,4.17
2020-03-04,1177,100.0,2.63,4.50,6.63,5.18,6.37,7.22,8.16,4.33,5.35,5.86,6.03,5.95,5.35,4.42,5.86
2020-03-05,1180,100.0,2.88,4.66,4.58,6.02,5.17,6.53,6.27,5.34,4.92,4.92,4.49,4.66,4.92,5.17,3.73
2020-03-06,1181,100.0,1.52,3.56,5.67,4.66,7.28,6.52,6.69,3.47,5.50,5.08,5.17,4.49,4.83,4.49,4.83
2020-03-07,1183,100.0,2.03,3.97,4.73,4.31,5.83,6.51,4.56,5.41,5.33,5.66,4.40,4.65,4.23,4.90,4.82
2020-03-08,1185,100.0,1.77,4.14,5.06,5.91,5.49,6.67,5.15,5.15,4.56,5.74,4.47,4.73,3.88,4.22,5.15
2020-03-09,1187,100.0,1.85,4.63,3.88,4.47,5.39,6.66,6.15,4.89,5.64,5.98,5.39,4.21,4.04,4.04,4.97
